In [1]:
import numpy as np
from scipy.stats import unitary_group
from numpy import linalg as LA
from scipy.linalg import block_diag
from scipy.linalg import logm, expm
import qiskit

In [2]:
#parameters and functions

d=5

def vvector(w,U):
    gamma0=np.diag(w)
    Udag=np.conjugate(U.T)
    gamma=np.matmul(U,np.matmul(gamma0,Udag))
    block1=gamma[0:3,0:3]
    block2=gamma[3:5,3:5]
    block1ev=LA.eigvals(block1)
    block2ev=LA.eigvals(block2)
    v=np.array([block1ev[0].real,block1ev[1].real,block1ev[2].real,block2ev[0].real,block2ev[1].real])
    return(v)

def conjvvector(w):
    word = np.sort(w)
    appo = (word[0]+word[4])/2
    eppo = (word[1]+word[3])/2
    v = np.array([appo,appo,eppo,eppo,word[2]])
    return(v)

def firstmajorizessecond(a,b):
    d=len(a)
    delta = np.absolute(sum(a)-sum(b))
    if delta>1e-10:
        print("comparing two vectors with different trace, diff = ",delta)
        return(False)
    aord=np.sort(a)[::-1]
    bord=np.sort(b)[::-1]
    acum = np.cumsum(aord)
    bcum = np.cumsum(bord)
    diff=acum-bcum
    diff[d-1]=0. #to avoid numerical errors
    if np.all(diff>=0):
        return(True)
    return(False)

def h(xx):
    x=np.real(xx)
    return -x*np.log2(x)-(1-x)*np.log2(1-x)

def H(v):
    summ=0
    d=len(v)
    for j in range(d):
        summ=summ+h(v[j])
    return summ

In [3]:
#define a w that you like
w=np.random.rand(d)
w=np.array([0.1,0.11,0.12,0.4,0.8])  

In [17]:
#code

countmax=2000000
count=0
eps=0.000005

U=np.identity(d,dtype=complex)
v=np.zeros(d)
conjv=np.zeros(d)
deltaH=0.

while(count<countmax):
    count=count+1
    w=np.random.rand(d) #if you want to keep changing w
    U=unitary_group.rvs(d)
    v=vvector(w,U)
    conjv=conjvvector(w)
    if(firstmajorizessecond(v,conjv)==False):
        print("a conjv does not majorize a corresponding v!")
    deltaH = H(conjv)-H(v)    
    if(deltaH <= 0):
        print("H(v) > H(vconj)!!!")
    if(deltaH <= eps):
        print("H(vconj) - H(v) = ", deltaH)
        print("vconj:", np.sort(conjv))
        print("v:", np.sort(v))